# Scapy
- https://thepacketgeek.com/
- http://scapy.readthedocs.io/en/latest/usage.html
Scapy can be run in two different modes, interactively as in these notbook demos or from a terminal window and programmatically from a Python script.

In [ ]:
from scapy.all import *

In [ ]:
lsc()

In [ ]:
ls()

### Most common commands
- lsc() - list all the scapy commands
- ls() - list all the supported protocols
- send(), - sniff(), - sr*() 

In [ ]:
# count argument value will sniff that many packets
pkt = sniff(count=2)

In [ ]:
type(pkt)

In [ ]:
pkt

In [ ]:
pkt[0].summary()

In [ ]:
ls(Ether)

In [ ]:
ls()

In [ ]:
ls(IP)

In [ ]:
ls(UDP)

In [31]:
ls(TCP)

sport      : ShortEnumField       = (20)
dport      : ShortEnumField       = (80)
seq        : IntField             = (0)
ack        : IntField             = (0)
dataofs    : BitField             = (None)
reserved   : BitField             = (0)
flags      : FlagsField           = (2)
window     : ShortField           = (8192)
chksum     : XShortField          = (None)
urgptr     : ShortField           = (0)
options    : TCPOptionsField      = ({})


## Under the Hood
Scapy uses Python dictionaries as the data structures for packets. Each packet is a collection of nested dictionaries with each layer being a child dictionary of the previous layer, built from the lowest layer up. E.g.
Ether | IP | ICMP <br><br>
Each field (such as the Ethernet ‘dst’ value or ICMP ‘type’ value) is a key:value pair in the appropriate layer. These fields (and nested layers) are all mutable so we can reassign them in place using the assignment operator.

In [32]:
ip = IP()
ip.show()


###[ IP ]###
  version= 4
  ihl= None
  tos= 0x0
  len= None
  id= 1
  flags= 
  frag= 0
  ttl= 64
  proto= hopopt
  chksum= None
  src= 127.0.0.1
  dst= 127.0.0.1
  \options\


In [ ]:
ip.src = "192.168.47.100" # some spoofed IP?
ip.ttl = 32 # hop limit that limits the lifespan or lieftime of packet in network
ip.show()


## Packet summary and show method
- summary() gives packet summary
- show() method shows detail packet contents

In [33]:
pkts = sniff(count=50)
pkts

<Sniffed: TCP:28 UDP:0 ICMP:20 Other:2>

In [34]:
pkts[0].summary()

'Ether / IP / TCP 127.0.0.1:46361 > 127.0.0.1:48226 A'

In [35]:
pkts[0].show()

###[ Ethernet ]###
  dst= 00:00:00:00:00:00
  src= 00:00:00:00:00:00
  type= 0x800
###[ IP ]###
     version= 4
     ihl= 5
     tos= 0x0
     len= 52
     id= 33257
     flags= DF
     frag= 0
     ttl= 64
     proto= tcp
     chksum= 0xbad8
     src= 127.0.0.1
     dst= 127.0.0.1
     \options\
###[ TCP ]###
        sport= 46361
        dport= 48226
        seq= 2069513005
        ack= 1805720462
        dataofs= 8
        reserved= 0
        flags= A
        window= 643
        chksum= 0xfe28
        urgptr= 0
        options= [('NOP', None), ('NOP', None), ('Timestamp', (5414646, 5414635))]


## Digging into Pakets by Layer

In [36]:
pkts[0]['Ethernet'].show()

###[ Ethernet ]###
  dst= 00:00:00:00:00:00
  src= 00:00:00:00:00:00
  type= 0x800
###[ IP ]###
     version= 4
     ihl= 5
     tos= 0x0
     len= 52
     id= 33257
     flags= DF
     frag= 0
     ttl= 64
     proto= tcp
     chksum= 0xbad8
     src= 127.0.0.1
     dst= 127.0.0.1
     \options\
###[ TCP ]###
        sport= 46361
        dport= 48226
        seq= 2069513005
        ack= 1805720462
        dataofs= 8
        reserved= 0
        flags= A
        window= 643
        chksum= 0xfe28
        urgptr= 0
        options= [('NOP', None), ('NOP', None), ('Timestamp', (5414646, 5414635))]


In [38]:
pkts[0]['Ethernet']['IP'].show()

###[ IP ]###
  version= 4
  ihl= 5
  tos= 0x0
  len= 52
  id= 33257
  flags= DF
  frag= 0
  ttl= 64
  proto= tcp
  chksum= 0xbad8
  src= 127.0.0.1
  dst= 127.0.0.1
  \options\
###[ TCP ]###
     sport= 46361
     dport= 48226
     seq= 2069513005
     ack= 1805720462
     dataofs= 8
     reserved= 0
     flags= A
     window= 643
     chksum= 0xfe28
     urgptr= 0
     options= [('NOP', None), ('NOP', None), ('Timestamp', (5414646, 5414635))]


In [39]:
pkts[0]['Ethernet']['IP']['TCP']

<TCP  sport=46361 dport=48226 seq=2069513005 ack=1805720462 dataofs=8 reserved=0 flags=A window=643 chksum=0xfe28 urgptr=0 options=[('NOP', None), ('NOP', None), ('Timestamp', (5414646, 5414635))] |>

In [40]:
pkts[0]['IP'].ttl

64

In [42]:
pkts[30]['ICMP'].type

8

## Packet .command() Method
- .command() method retuns a string of the command necessary to recreate that packet

In [43]:
pkts[2].command()

"Ether(dst='00:00:00:00:00:00', src='00:00:00:00:00:00', type=34525)/IPv6(version=6, tc=0, fl=705509, plen=32, nh=6, hlim=64, src='::1', dst='::1')/TCP(sport=58804, dport=8888, seq=950798151, ack=1654315182, dataofs=8, reserved=0, flags=16, window=3636, chksum=40, urgptr=0, options=[('NOP', None), ('NOP', None), ('Timestamp', (5414648, 5414637))])"

In [44]:
newPkt = pkts[2]
# or 
# newPkt = eval(pkts[2].command())

In [45]:
newPkt

<Ether  dst=00:00:00:00:00:00 src=00:00:00:00:00:00 type=0x86dd |<IPv6  version=6 tc=0 fl=705509 plen=32 nh=TCP hlim=64 src=::1 dst=::1 |<TCP  sport=58804 dport=8888 seq=950798151 ack=1654315182 dataofs=8 reserved=0 flags=A window=3636 chksum=0x28 urgptr=0 options=[('NOP', None), ('NOP', None), ('Timestamp', (5414648, 5414637))] |>>>

## Configuration
- conf - global conf object that has all the configuration relevant to scapy

In [46]:
conf

ASN1_default_codec = <ASN1Codec BER[1]>
AS_resolver = <scapy.as_resolvers.AS_resolver_multi object at 0x7f3ba613e048>
BTsocket   = <BluetoothL2CAPSocket: read/write packets on a connected L2CAP ...
L2listen   = <L2ListenSocket: read packets at layer 2 using Linux PF_PACKET ...
L2socket   = <L2Socket: read/write packets at layer 2 using Linux PF_PACKET ...
L3socket   = <L3PacketSocket: read/write packets at layer 3 using Linux PF_P...
auto_fragment = 1
checkIPID  = 0
checkIPaddr = 1
checkIPsrc = 1
check_TCPerror_seqack = 0
color_theme = <RastaTheme>
commands   = arpcachepoison : Poison target's cache with (your MAC,victim's ...
debug_dissector = 0
debug_match = 0
default_l2 = <class 'scapy.packet.Raw'>
emph       = <Emphasize []>
ethertypes = </etc/ethertypes/ >
except_filter = ''
extensions_paths = '.'
histfile   = '/root/.scapy_history'
iface      = 'eth0'
iface6     = 'lo'
interactive = False
interactive_shell = 'ipython'
ipv6_enabled = True
ipython_embedded = True
l2types    = 0x1 <

In [47]:
conf.verb=1
conf.color_theme = RastaTheme()

In [48]:
ls()

AH         : AH
ARP        : ARP
ASN1_Packet : None
BOOTP      : BOOTP
CookedLinux : cooked linux
DHCP       : DHCP options
DHCP6      : DHCPv6 Generic Message)
DHCP6OptAuth : DHCP6 Option - Authentication
DHCP6OptBCMCSDomains : DHCP6 Option - BCMCS Domain Name List
DHCP6OptBCMCSServers : DHCP6 Option - BCMCS Addresses List
DHCP6OptClientFQDN : DHCP6 Option - Client FQDN
DHCP6OptClientId : DHCP6 Client Identifier Option
DHCP6OptDNSDomains : DHCP6 Option - Domain Search List option
DHCP6OptDNSServers : DHCP6 Option - DNS Recursive Name Server
DHCP6OptElapsedTime : DHCP6 Elapsed Time Option
DHCP6OptGeoConf : 
DHCP6OptIAAddress : DHCP6 IA Address Option (IA_TA or IA_NA suboption)
DHCP6OptIAPrefix : DHCP6 Option - IA_PD Prefix option
DHCP6OptIA_NA : DHCP6 Identity Association for Non-temporary Addresses Option
DHCP6OptIA_PD : DHCP6 Option - Identity Association for Prefix Delegation
DHCP6OptIA_TA : DHCP6 Identity Association for Temporary Addresses Option
DHCP6OptIfaceId : DHCP6 Interface-

In [49]:
ip

<IP  |>

## Using Scapy with Python
- more detail examples later

In [57]:
count = 0
for packet in pkts:
    #print(packet.summary())
    if (packet.haslayer(ICMP)):
        count += 1
        print("ICMP code: "+ str(packet.getlayer(ICMP).code))
    else:
        print(packet.summary())
print('Total ICMP packets: {}'.format(count))

Ether / IP / TCP 127.0.0.1:46361 > 127.0.0.1:48226 A
Ether / IP / TCP 127.0.0.1:46361 > 127.0.0.1:48226 A
Ether / IPv6 / TCP ::1:58804 > ::1:8888 A
Ether / IPv6 / TCP ::1:58804 > ::1:8888 A
Ether / IPv6 / TCP ::1:8888 > ::1:58804 PA / Raw
Ether / IPv6 / TCP ::1:8888 > ::1:58804 PA / Raw
Ether / IPv6 / TCP ::1:58804 > ::1:8888 A
Ether / IPv6 / TCP ::1:58804 > ::1:8888 A
Ether / IPv6 / TCP ::1:59280 > ::1:8888 PA / Raw
Ether / IPv6 / TCP ::1:59280 > ::1:8888 PA / Raw
Ether / IPv6 / TCP ::1:8888 > ::1:59280 A
Ether / IPv6 / TCP ::1:8888 > ::1:59280 A
Ether / IPv6 / TCP ::1:8888 > ::1:59280 PA / Raw
Ether / IPv6 / TCP ::1:8888 > ::1:59280 PA / Raw
Ether / IPv6 / TCP ::1:59280 > ::1:8888 A
Ether / IPv6 / TCP ::1:59280 > ::1:8888 A
ICMP code: 0
ICMP code: 0
ICMP code: 0
ICMP code: 0
Ether / ARP who has 192.168.231.2 says 192.168.231.131
Ether / ARP is at 00:50:56:f9:21:6d says 192.168.231.2 / Padding
ICMP code: 0
ICMP code: 0
ICMP code: 0
ICMP code: 0
ICMP code: 0
ICMP code: 0
ICMP code: 0
I